In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import os, cv2
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

/home/santhanalakshmi/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/santhanalakshmi/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/santhanalakshmi/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/santhanalakshmi/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/santhanalakshmi/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/santhanalakshmi

In [5]:
data = []
def create_data():
    image_size = 150
    for item in ['normal', 'pneumonia']:
        path = "/home/santhanalakshmi/pneumonia_dataset/train/" + item + "/"
        for img in os.listdir(path):
            image = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            image_resize = cv2.resize(image, (image_size, image_size))
            if item == 'normal':
                data.append([image_resize, 0])
            else:
                data.append([image_resize, 1])
    print(len(data))
create_data() 

2425


In [6]:
np.random.shuffle(data)
x = []
y = []
for image in data:
    x.append(image[0])
    y.append(image[1])
x = np.asarray(x)
y = np.asarray(y)
print(x.shape, '', y.shape)

(2425, 150, 150)  (2425,)


In [9]:
x = x.reshape(-1, 150, 150, 1)
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3, random_state = 42)

In [21]:
from keras.models import Sequential
model = Sequential([
    Conv2D(filters=32, kernel_size = (3,3), activation='relu', input_shape = X_train[0].shape),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    
    Flatten(),
    Dense(64, activation='relu'),
    Dense(units=1, activation='sigmoid')  
])

In [22]:
model.compile(optimizer='adam',
                     loss='binary_crossentropy',
                     metrics=['accuracy'])


In [27]:
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
1697/1697 [==============================] - 118s 70ms/step - loss: 0.3359 - acc: 0.8456
Epoch 2/10
1697/1697 [==============================] - 119s 70ms/step - loss: 0.2384 - acc: 0.9010
Epoch 3/10
1697/1697 [==============================] - 114s 67ms/step - loss: 0.1706 - acc: 0.9358
Epoch 4/10
1697/1697 [==============================] - 116s 68ms/step - loss: 0.1174 - acc: 0.9664
Epoch 5/10
1697/1697 [==============================] - 115s 68ms/step - loss: 0.0691 - acc: 0.9829
Epoch 6/10
1697/1697 [==============================] - 115s 68ms/step - loss: 0.0450 - acc: 0.9959
Epoch 7/10
1697/1697 [==============================] - 118s 70ms/step - loss: 0.0275 - acc: 0.9988
Epoch 8/10
1697/1697 [==============================] - 116s 69ms/step - loss: 0.0136 - acc: 0.9994
Epoch 9/10
1697/1697 [==============================] - 117s 69ms/step - loss: 0.0090 - acc: 0.9994
Epoch 10/10
1697/1697 [==============================] - 116s 69ms/step - loss: 0.0056 - acc: 1.0000

In [29]:
model.evaluate(X_val, y_val)

728/728 [==============================] - 12s 16ms/step


[1.287813554753314, 0.5810439553889599]

TypeError: 'History' object is not callable

In [34]:
test_image_order = pd.read_csv("/home/santhanalakshmi/pneumonia_dataset/test.csv")
print(test_image_order.head())

file_paths = [[fname, '/home/santhanalakshmi/pneumonia_dataset/test/' + fname] for fname in test_image_order['filename']]
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
print(test_images.head())

test_pixel_data = []    
for i in range(len(test_images)):
    img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE)
    new_img_array =cv2.resize(img_array,(150,150))
    test_pixel_data.append(new_img_array)
test_pixel_data = np.asarray(test_pixel_data)
test_pixel_data =  test_pixel_data.reshape(-1, 150, 150, 1)
pred = model.predict(test_pixel_data)
predictions = []
for item in pred:
  if item <= 0.5:
    predictions.append('normal')
  else:
    predictions.append('pneumonia')
res = pd.DataFrame({'filename': test_images['filename'], 'label': predictions}) 
res.to_csv("submission.csv", index = False) 

           filename
0  CXR_test_519.png
1  CXR_test_578.png
2  CXR_test_359.png
3  CXR_test_573.png
4  CXR_test_471.png
           filename                                          filepaths
0  CXR_test_519.png  /home/santhanalakshmi/pneumonia_dataset/test/C...
1  CXR_test_578.png  /home/santhanalakshmi/pneumonia_dataset/test/C...
2  CXR_test_359.png  /home/santhanalakshmi/pneumonia_dataset/test/C...
3  CXR_test_573.png  /home/santhanalakshmi/pneumonia_dataset/test/C...
4  CXR_test_471.png  /home/santhanalakshmi/pneumonia_dataset/test/C...


In [35]:
sub = pd.read_csv("submission.csv")
sub.head()

,filename,label
0,CXR_test_519.png,pneumonia
1,CXR_test_578.png,normal
2,CXR_test_359.png,pneumonia
3,CXR_test_573.png,pneumonia
4,CXR_test_471.png,pneumonia


In [36]:
sub.label.value_counts()

pneumonia    385
normal       221
Name: label, dtype: int64